# 40 - Experiment #2

## Initialization

In [43]:
import os, sys
from pathlib import Path

ENV_IS_GOOGLE_COLAB = True if 'google.colab' in str(get_ipython()) else False
os.environ["ENV_IS_GOOGLE_COLAB"] = str(ENV_IS_GOOGLE_COLAB)

if ENV_IS_GOOGLE_COLAB:
  from google.colab import drive
  mountpoint = Path("/content/drive")
  drive.mount(str(mountpoint)) # Mount gdrive to BASE
  base = mountpoint / "MyDrive/SODUCO/article_das_2022" # Adapt this to your situation
  sys.path.append(str(base)) # Add BASE to Python Path
else:
  base = os.path.dirname(os.path.realpath("__file__")) # If not on GColab, BASE will be the directory of this notebook

BASE = Path(base).resolve() # Make BASE absolute
print(sys.path)
BASE

['/home/bertrand/dev/paper-ner-bench-das22/src/ner', '/home/bertrand/anaconda3/lib/python39.zip', '/home/bertrand/anaconda3/lib/python3.9', '/home/bertrand/anaconda3/lib/python3.9/lib-dynload', '', '/home/bertrand/anaconda3/lib/python3.9/site-packages', '/home/bertrand/anaconda3/lib/python3.9/site-packages/locket-0.2.1-py3.9.egg', '/home/bertrand/anaconda3/lib/python3.9/site-packages/IPython/extensions', '/home/bertrand/.ipython', '/home/bertrand/.cache/huggingface/modules']


PosixPath('/home/bertrand/dev/paper-ner-bench-das22/src/ner')

In [44]:
!pip install --upgrade transformers datasets transformers[sentencepiece] seqeval

In [45]:
# Same training configuration as in 20-experiment_1
TRAINING_CONFIG = {
    "evaluation_strategy": "steps",
    "eval_steps": 100,
    "max_steps": 5000,
    "learning_rate": 1e-4,
    "per_device_train_batch_size": 16,
    "per_device_eval_batch_size": 16,
    "weight_decay": 1e-5,
    "load_best_model_at_end": True,
    "greater_is_better":True,
    "metric_for_best_model": "f1",
    "save_strategy": "steps",
    "save_steps": 100, # Make Early callback bug ?
    "save_total_limit": 1,
}

# Training set to fine-tune the NER layer
REF_GOLD_DATASET = BASE / "02-experiment_2_prepared_datasets/huggingface_ref"
PERO_GOLD_DATASET = BASE / "02-experiment_2_prepared_datasets/huggingface_pero"
assert REF_GOLD_DATASET.exists()
assert PERO_GOLD_DATASET.exists()

REF_GOLD_DATASET, PERO_GOLD_DATASET

(PosixPath('/home/bertrand/dev/paper-ner-bench-das22/src/ner/02-experiment_2_prepared_datasets/huggingface_ref'),
 PosixPath('/home/bertrand/dev/paper-ner-bench-das22/src/ner/02-experiment_2_prepared_datasets/huggingface_pero'))

## 41. Fine-tuning CamemBERT-pretrained for NER on the REFERENCE gold set (manually corrected entries)

In [ ]:
from datasets import load_from_disk
from camembert_util import train_eval_loop, init_model

# PATHS
MODEL_NAME = BASE / "10-camembert_pretrained_model"

train_dev_test = load_from_disk(REF_GOLD_DATASET)

local_config = TRAINING_CONFIG.copy()
local_config["output_dir"] = BASE / "41-camembert_pretrained_finetuned_ref"

# Get the model components
model, tokenizer, training_args = init_model(str(MODEL_NAME), local_config)

# Run train eval
metrics = train_eval_loop(model,
                          training_args, 
                          tokenizer,     
                          **train_dev_test, 
                          patience=5)
metrics

17/01/2022 09:56:35 ; INFO ; Model /home/bertrand/dev/paper-ner-bench-das22/src/ner/10-camembert_pretrained_model
Didn't find file /home/bertrand/dev/paper-ner-bench-das22/src/ner/10-camembert_pretrained_model/added_tokens.json. We won't load it.
loading file /home/bertrand/dev/paper-ner-bench-das22/src/ner/10-camembert_pretrained_model/sentencepiece.bpe.model
loading file /home/bertrand/dev/paper-ner-bench-das22/src/ner/10-camembert_pretrained_model/tokenizer.json
loading file None
loading file /home/bertrand/dev/paper-ner-bench-das22/src/ner/10-camembert_pretrained_model/special_tokens_map.json
loading file /home/bertrand/dev/paper-ner-bench-das22/src/ner/10-camembert_pretrained_model/tokenizer_config.json


TypeError: __init__() missing 1 required positional argument: 'output_dir'

Run the following if you want to download the model from GColab. It might take a while.

In [ ]:
if ENV_IS_GOOGLE_COLAB:
  !zip -r "/tmp/41-experiment_2_finetuned_camembert_pretrained.zip" $FINETUNED_MODEL_OUTPUT_DIR

  from google.colab import files
  files.download("/tmp/41-experiment_2_finetuned_camembert_pretrained.zip")

## 42 - Fine-tuning CamemBERT for NER on the REFERENCE gold set (manually corrected entries)

In [ ]:
from datasets import load_from_disk
from camembert_util import train_eval_loop, init_model

MODEL_NAME = "Jean-Baptiste/camembert-ner"

local_config = TRAINING_CONFIG.copy()
local_config["output_dir"] = BASE / "42-camembert_finetuned_ref" 

train_dev_test = load_from_disk(REF_GOLD_DATASET)

# Get the model components
model, tokenizer, training_args = init_model(str(MODEL_NAME), local_config)

# Run train eval
metrics = train_eval_loop(model,
                          training_args, 
                          tokenizer,     
                          **train_dev_test,
                          patience=5)
metrics

## 43 - Fine-tuning CamemBERT-pretrained for NER on the PERO-OCR gold set (noisy data)

In [ ]:
from datasets import load_from_disk
from camembert_util import train_eval_loop, init_model

MODEL_NAME = "HueyNemud/das22-10-camembert_pretrained"

local_config = TRAINING_CONFIG.copy()
local_config["output_dir"] = BASE / "43-camembert_pretrained_finetuned_pero"

train_dev_test = load_from_disk(PERO_GOLD_DATASET)

# Get the model components
model, tokenizer, training_args = init_model(str(MODEL_NAME), local_config)

# Run train eval
metrics = train_eval_loop(model,
                          training_args, 
                          tokenizer,     
                          **train_dev_test,
                          patience=5)
metrics

## 44 - Fine-tuning CamemBERT for NER on the PERO-OCR gold set (noisy data)

In [5]:
from datasets import load_from_disk
from camembert_util import train_eval_loop, init_model

MODEL_NAME = "Jean-Baptiste/camembert-ner"

local_config = TRAINING_CONFIG.copy()
local_config["output_dir"] = BASE / "44-camembert_finetuned_pero"

train_dev_test = load_from_disk(PERO_GOLD_DATASET)

# Get the model components
model, tokenizer, training_args = init_model(str(MODEL_NAME), local_config)

# Run train eval
metrics = train_eval_loop(model,
                          training_args, 
                          tokenizer,     
                          **train_dev_test,
                          patience=5)
metrics

18/01/2022 03:18:28 ; INFO ; Model Jean-Baptiste/camembert-ner


Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at Jean-Baptiste/camembert-ner and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([5, 768]) in the checkpoint and torch.Size([9, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([5]) in the checkpoint and torch.Size([9]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
max_steps is given, it will override any value given in num_train_epochs
The following columns in the training set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: tokens, ner_tags.
***** Running training *****
  Num examples = 6004
  Num Epochs = 14
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optim

Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
100,No log,0.247325,0.912333,0.925744,0.918990,0.960895
200,No log,0.171694,0.930514,0.928760,0.929636,0.963143
300,No log,0.132045,0.935375,0.943837,0.939587,0.970515
400,No log,0.142074,0.936490,0.939314,0.937900,0.968141
500,0.245500,0.105783,0.949794,0.955522,0.952649,0.975887
600,0.245500,0.107380,0.948120,0.950622,0.949369,0.973138
700,0.245500,0.111011,0.951504,0.954014,0.952757,0.972889
800,0.245500,0.093002,0.947999,0.955145,0.951558,0.975387
900,0.245500,0.100926,0.956193,0.954391,0.955291,0.976262
1000,0.086800,0.090775,0.955263,0.957784,0.956522,0.976637


The following columns in the evaluation set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: tokens, ner_tags.
***** Running Evaluation *****
  Num examples = 668
  Batch size = 16


Downloading:   0%|          | 0.00/2.48k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to /content/drive/MyDrive/SODUCO/article_das_2022/44-camembert_finetuned_pero/checkpoint-100
Configuration saved in /content/drive/MyDrive/SODUCO/article_das_2022/44-camembert_finetuned_pero/checkpoint-100/config.json
Model weights saved in /content/drive/MyDrive/SODUCO/article_das_2022/44-camembert_finetuned_pero/checkpoint-100/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/SODUCO/article_das_2022/44-camembert_finetuned_pero/checkpoint-100/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/SODUCO/article_das_2022/44-camembert_finetuned_pero/checkpoint-100/special_tokens_map.json
The following columns in the evaluation set  

The following columns in the evaluation set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: tokens, ner_tags.
***** Running Evaluation *****
  Num examples = 668
  Batch size = 16


({'epoch': 6.65,
  'eval_accuracy': 0.9386906641406167,
  'eval_f1': 0.9364307016167006,
  'eval_loss': 0.2547541558742523,
  'eval_precision': 0.9307332293291731,
  'eval_recall': 0.9421983575489576,
  'eval_runtime': 6.0384,
  'eval_samples_per_second': 276.396,
  'eval_steps_per_second': 17.389},
 {'epoch': 6.65,
  'eval_accuracy': 0.9777611194402799,
  'eval_f1': 0.9612490594431903,
  'eval_loss': 0.09422445297241211,
  'eval_precision': 0.9594442358242583,
  'eval_recall': 0.9630606860158312,
  'eval_runtime': 2.6655,
  'eval_samples_per_second': 250.608,
  'eval_steps_per_second': 15.757})

## 45 - Evaluate CamemBERT-pretrained and camembert (simple) on the REF, PERO and TESS test datasets

Load the model fine-tuned on the ref data from `BASE/41-experiment_2_finetuned_camembert_pretrained`.

If you prefer to use the model shared on huggingface.co, replace `model_path` with `HueyNemud/HueyNemud/das2022-41-berties-pretrained-finetuned-ref`. 


In [49]:
REF_GOLD_DATASET = BASE / "02-experiment_2_prepared_datasets/huggingface_ref"
PERO_GOLD_DATASET = BASE / "02-experiment_2_prepared_datasets/huggingface_pero"
TESS_GOLD_DATASET = BASE / "02-experiment_2_prepared_datasets/huggingface_tess"

REF_GOLD_DATASET, PERO_GOLD_DATASET, TESS_GOLD_DATASET 

(PosixPath('/home/bertrand/dev/paper-ner-bench-das22/src/ner/02-experiment_2_prepared_datasets/huggingface_ref'),
 PosixPath('/home/bertrand/dev/paper-ner-bench-das22/src/ner/02-experiment_2_prepared_datasets/huggingface_pero'),
 PosixPath('/home/bertrand/dev/paper-ner-bench-das22/src/ner/02-experiment_2_prepared_datasets/huggingface_tess'))

In [50]:
# Where to store the computed metrics
METRICS_OUTPUT = BASE / "45-experiment_2_metrics"
METRICS_OUTPUT.mkdir(exist_ok=True, parents=True)
METRICS_OUTPUT

PosixPath('/home/bertrand/dev/paper-ner-bench-das22/src/ner/45-experiment_2_metrics')

In [51]:
#Helper func to predict & save the computed metrics
import json
from datasets import load_from_disk

def eval_save(dataset_path, name):
    train_dev_test = load_from_disk(dataset_path)
    predictions_ref = trainer.predict(train_dev_test["test"])

    with open( METRICS_OUTPUT / f"camembert_{name}.json" ,"w", encoding="utf-8") as pf:
        js = json.dumps(predictions_ref.metrics)
        pf.write(js)
        js

### Evaluate 41-camembert_pretrained_finetuned_ref

In [ ]:
# We won't train so keep it minimalistic
dummy_config = {"output_dir": "/tmp"}

# Load the fine-tuned model
from transformers import AutoModelForTokenClassification, DataCollatorForTokenClassification, Trainer
from camembert_util import compute_metrics, init_model


model_path = BASE / "41-camembert_pretrained_finetuned_ref"
model, tokenizer, training_args = init_model(str(model_path), dummy_config)

model.eval() # Switch to evaluation mode

data_collator = DataCollatorForTokenClassification(tokenizer)
trainer = Trainer(
    model,
    training_args,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
eval_save(REF_GOLD_DATASET, "pretrained_ref_test_ref")
eval_save(PERO_GOLD_DATASET, "pretrained_ref_test_pero")
eval_save(TESS_GOLD_DATASET, "pretrained_ref_test_tess")

### Evaluate 42-camembert_finetuned_ref

In [ ]:
# Load the fine-tuned model
from transformers import AutoModelForTokenClassification, DataCollatorForTokenClassification, Trainer
from camembert_util import compute_metrics

# We won't train so keep it minimalistic
dummy_config = {"output_dir": "/tmp"}

model_path = BASE / "42-camembert_finetuned_ref"
model, tokenizer, training_args = init_model(str(model_path), dummy_config)

model.eval() # Switch to evaluation mode

data_collator = DataCollatorForTokenClassification(tokenizer)
trainer = Trainer(
    model,
    training_args,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

16/01/2022 11:29:13 ; INFO ; Model /home/bertrand/dev/paper-ner-bench-das22/src/ner/42-experiment_2_finetuned_camembert
Didn't find file /home/bertrand/dev/paper-ner-bench-das22/src/ner/42-experiment_2_finetuned_camembert/added_tokens.json. We won't load it.
loading file /home/bertrand/dev/paper-ner-bench-das22/src/ner/42-experiment_2_finetuned_camembert/sentencepiece.bpe.model
loading file /home/bertrand/dev/paper-ner-bench-das22/src/ner/42-experiment_2_finetuned_camembert/tokenizer.json
loading file None
loading file /home/bertrand/dev/paper-ner-bench-das22/src/ner/42-experiment_2_finetuned_camembert/special_tokens_map.json
loading file /home/bertrand/dev/paper-ner-bench-das22/src/ner/42-experiment_2_finetuned_camembert/tokenizer_config.json
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start upd

In [ ]:
eval_save(REF_GOLD_DATASET, "ref_test_ref")
eval_save(PERO_GOLD_DATASET, "ref_test_pero")
eval_save(TESS_GOLD_DATASET, "ref_test_tess")

The following columns in the test set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: tokens, ner_tags.
***** Running Prediction *****
  Num examples = 1669
  Batch size = 8


The following columns in the test set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: tokens, ner_tags.
***** Running Prediction *****
  Num examples = 1669
  Batch size = 8
The following columns in the test set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: tokens, ner_tags.
***** Running Prediction *****
  Num examples = 1669
  Batch size = 8


### Evaluate 43-camembert_pretrained_finetuned_pero

In [ ]:
from transformers import AutoModelForTokenClassification, DataCollatorForTokenClassification, Trainer
from camembert_util import compute_metrics

# We won't train so keep it minimalistic
dummy_config = {"output_dir": "/tmp"}

model_path = BASE / "43-camembert_pretrained_finetuned_pero"
model, tokenizer, training_args = init_model(str(model_path), dummy_config)

model.eval() # Switch to evaluation mode

data_collator = DataCollatorForTokenClassification(tokenizer)
trainer = Trainer(
    model,
    training_args,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
eval_save(REF_GOLD_DATASET, "pretrained_pero_test_ref")
eval_save(PERO_GOLD_DATASET, "pretrained_pero_test_pero")
eval_save(TESS_GOLD_DATASET, "pretrained_pero_test_tess")

### Evaluate 44-camembert_finetuned_pero

In [47]:
# Load the fine-tuned model
from transformers import AutoModelForTokenClassification, DataCollatorForTokenClassification, Trainer
from camembert_util import compute_metrics

# We won't train so keep it minimalistic
dummy_config = {"output_dir": "/tmp"}

model_path = BASE / "44-camembert_finetuned_pero"
model, tokenizer, training_args = init_model(str(model_path), dummy_config)

model.eval() # Switch to evaluation mode

data_collator = DataCollatorForTokenClassification(tokenizer)
trainer = Trainer(
    model,
    training_args,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

18/01/2022 04:53:26 ; INFO ; Model /home/bertrand/dev/paper-ner-bench-das22/src/ner/44-camembert_finetuned_pero
Didn't find file /home/bertrand/dev/paper-ner-bench-das22/src/ner/44-camembert_finetuned_pero/added_tokens.json. We won't load it.
loading file /home/bertrand/dev/paper-ner-bench-das22/src/ner/44-camembert_finetuned_pero/sentencepiece.bpe.model
loading file /home/bertrand/dev/paper-ner-bench-das22/src/ner/44-camembert_finetuned_pero/tokenizer.json
loading file None
loading file /home/bertrand/dev/paper-ner-bench-das22/src/ner/44-camembert_finetuned_pero/special_tokens_map.json
loading file /home/bertrand/dev/paper-ner-bench-das22/src/ner/44-camembert_finetuned_pero/tokenizer_config.json
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-)

In [52]:
eval_save(REF_GOLD_DATASET, "pero_test_ref")
eval_save(PERO_GOLD_DATASET, "pero_test_pero")
eval_save(TESS_GOLD_DATASET, "pero_test_tess")

The following columns in the test set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: tokens, ner_tags.
***** Running Prediction *****
  Num examples = 1669
  Batch size = 8
The following columns in the test set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: tokens, ner_tags.
***** Running Prediction *****
  Num examples = 1669
  Batch size = 8
The following columns in the test set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: tokens, ner_tags.
***** Running Prediction *****
  Num examples = 1669
  Batch size = 8
